# Geocue tutorial scraping

In [10]:
dev = False # Enable to run and print test outputs below each function definition

In [34]:
from selenium import webdriver
import pyautogui
from bs4 import BeautifulSoup
import requests
import time

from webdriver_manager.firefox import GeckoDriverManager

In [3]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - Downloading: 19.0kB [00:00, 17.2MB/s]                   


## Function definitions

In [4]:
def finnish_to_english_keyboard(string):
    source_symbols = '-'
    target_symbols = '+'
    remove_symbols = r'\/:*?"<>|'
    translation = string.maketrans(source_symbols, target_symbols, remove_symbols)
    return string.translate(translation)

if dev:
    test_string = r'Sample title? - For testing \/'
    print('Before: ' + test_string)
    print('After: ' + finnish_to_english_keyboard(test_string))

Before: Sample title? - For testing \/
After: Sample title + For testing 


In [4]:
testurl = 'https://support.geocue.com/whats-new-in-terrasolid-mar-2022/'
#testurl = 'https://support.geocue.com/including-spot-elevations-cells-in-surface/'
testurl = 'https://support.geocue.com/zero-ground-points-classified/'
test_dir = 'scrape'

def finnish_to_english_keyboard(string):
    source_symbols = '-'
    target_symbols = '+'
    remove_symbols = r'\/:*?"<>|'
    translation = string.maketrans(source_symbols, target_symbols, remove_symbols)
    return string.translate(translation)

def scrape_url(driver, url, save_dir):
    driver.get(url)
    pyautogui.hotkey('ctrl', 's')

    #pyautogui.write(save_dir)
    # with pyautogui.hold('shift'):
    #     pyautogui.write('/')
    #pyautogui.write('-')

    title = str(driver.title)
    title = finnish_to_english_keyboard(title)
    #pyautogui.write(title)

    time.sleep(1)
    pyautogui.hotkey('enter')

    return True

if dev:
    scrape_url(driver, testurl, test_dir)

In [41]:
def remove_special_characters(string):
    source_symbols = ''
    target_symbols = ''
    remove_symbols = r'\/:*?"<>|'
    translation = string.maketrans(source_symbols, target_symbols, remove_symbols)
    return string.translate(translation)

def extract_title(page):
    soup = BeautifulSoup(page)
    title = soup.title.text
    simple_title = remove_special_characters(title)
    return simple_title

def scrape_simple_html(url):
    page = requests.get(url)
    title = extract_title(page.content)
    return page, title

if dev:
    testurl = 'https://support.geocue.com/tutorial-matching-mobile-data-xyz-using-terramatch/'
    page, title = scrape_simple_html(testurl)

'Tutorial Matching Mobile data XYZ using TerraMatch - GeoCue'

In [26]:
def html_to_file(page, fname, dir):
    with open(dir + fname + '.html', 'w', encoding='utf-8') as f:
        f.write(page.text)

if dev:
    testurl = 'https://support.geocue.com/tutorial-terrascan-adding-synthetic-points/'
    page, title = scrape_simple_html(testurl)
    html_to_file(page, title, '../../Downloads/tutorials/')

In [36]:
testurl = 'https://support.geocue.com/category/terrasolid/terrasolid-tools-tips-workflows/'
testurl = 'https://support.geocue.com/category/terrasolid/terrasolid-tutorials/'

def parse_articles(page_source):
    soup = BeautifulSoup(page_source)
    elements = soup.find_all('a', class_='btn btn-primary btn-block')

    for element in elements:
        link = element['href']

        yield link
    
if dev:
    testpage = requests.get(testurl).content
    for link in parse_articles(testpage):
        print(link)

https://support.geocue.com/tutorial-terrascan-adding-synthetic-points/
https://support.geocue.com/tutorial-matching-mobile-data-xyz-using-terramatch/
https://support.geocue.com/vtw-2022-terrasolid-software-licensing/
https://support.geocue.com/terrasolid-tutorial-coordinate-systems-projections/
https://support.geocue.com/terrasolid-tutorial-define-signal-markers/
https://support.geocue.com/terrasolid-tutorial-strip-adjustment-dji-l1/
https://support.geocue.com/terrasolid-tutorial-processing-dji-l1/
https://support.geocue.com/terrasolid-tutorial-check-building-footprint-polygons/
https://support.geocue.com/terrasolid-tutorial-vegetation-index-echo-length/
https://support.geocue.com/terrasolid-tutorial-digital-terrain-model/


In [6]:
def scrape_index(base_url, start_i, last_i):
    last_i += 1
    
    for i in range(start_i, last_i):
        target_url = base_url + str(i)

        page = requests.get(target_url).content
        for content in parse_articles(page):
            scrape_url(driver, content, '')

In [43]:
def scrape_index_htmlonly(base_url, start_i, last_i, save_dir='./'):
    last_i += 1
    
    enumerator = 0

    for i in range(start_i, last_i):
        target_url = base_url + str(i)

        page = requests.get(target_url).content
        for url in parse_articles(page):
            enumerator += 1
            page, title = scrape_simple_html(url)
            fname = str(enumerator) + ' ' + title
            html_to_file(page, fname, save_dir)

## Scraping full web pages with supporting files (css, images, etc.)

In [ ]:
########################################
# Terrasolid Tools, Tips and Workflows #
########################################

base_url = 'https://support.geocue.com/category/terrasolid/terrasolid-tools-tips-workflows/page/'

scrape_index(base_url, 1, 26)

In [25]:
##################################################
# Terrasolid Installation, Licensing and Updates #
##################################################

base_url = 'https://support.geocue.com/category/terrasolid/terrasolid-installation-licensing-updates/page/'

scrape_index(base_url, 1, 7)

## Scraping only html

Cause Youtube embeds are annoying (slowing full scrape a lot) and not really any valuable information is lost

In [44]:
########################
# Terrasolid Tutorials #
########################

base_url = 'https://support.geocue.com/category/terrasolid/terrasolid-tutorials/page/'

scrape_index_htmlonly(base_url, 1, 8, '../../Downloads/tutorials/')

In [ ]:
########################
# Terrasolid Tutorials #
########################

base_url = 'https://support.geocue.com/category/terrasolid/terrasolid-case-studies/page/'

scrape_index_htmlonly(base_url, 1, 1, '../../Downloads/case studies/')